In [1]:
%load_ext autoreload
%autoreload 2
import pygsti
import copy
from pygsti import tools as pgt
from pygsti import modelmembers as pgmm
from pygsti.processors import QubitProcessorSpec
from pygsti.models.explicitmodel import ExplicitOpModel
from pygsti.baseobjs.statespace import ExplicitStateSpace
from pygsti.modelmembers.povms import TPPOVM
from pygsti.protocols.gst import _add_gauge_opt, ModelEstimateResults
import numpy as np

In [2]:
class Infrastructure:

    @staticmethod
    def to_3level_unitary(U_2level):
        U_3level = np.zeros((3, 3), complex)
        U_3level[0:2, 0:2] = U_2level
        U_3level[2, 2] = 1.0
        return U_3level

    @staticmethod
    def unitary_to_gmgate(U):
        GU_std = pgt.unitary_to_std_process_mx(U)
        return pgt.change_basis(GU_std, "std", "gm")

    @staticmethod
    def leakage_friendly_basis():
        """ 
        Since the 3-level model's process matrices are in the Gell-Mann basis, let's change them over to
        another more convenient basis before we make the report.  This basis is used to isolate the 
        parts of Hilbert-Schmidt space that act on the computational subspace.
        """
        gm_basis = pygsti.baseobjs.Basis.cast("gm", 9)
        leakage_basis_mxs = [
            np.sqrt(2) / 3 * (np.sqrt(3) * gm_basis[0] + 0.5 * np.sqrt(6) * gm_basis[8]),
            gm_basis[1],
            gm_basis[4],
            gm_basis[7],
            gm_basis[2],
            gm_basis[3],
            gm_basis[5],
            gm_basis[6],
            1 / 3 * (np.sqrt(3) * gm_basis[0] - np.sqrt(6) * gm_basis[8]),
        ]
        check = np.zeros((9, 9), complex)
        for i, m1 in enumerate(leakage_basis_mxs):
            for j, m2 in enumerate(leakage_basis_mxs):
                check[i, j] = np.vdot(m1, m2)
        assert np.allclose(check, np.identity(9, complex))
        leakage_basis = pygsti.baseobjs.ExplicitBasis(
            leakage_basis_mxs,
            name="LeakageBasis",
            longname="2+1 level leakage basis",
            real=True,
            labels=["I", "X", "Y", "Z", "LX0", "LX1", "LY0", "LY1", "L"],
        )
        return leakage_basis

    @staticmethod
    def changebasis_3level_model(mdl, leakage_basis=None):
        """ 
        Create a copy of "mdl" where attached modelmembers are unconstrained
        and expressed in the leakage-friendly basis.

        This is needed because some modelmember classes (like TPPOVM) require
        that the identity matrix is an element of our basis for Hilbert-Schmidt
        space, and the leakage-friendly basis doesn't have that property.
        """
        if leakage_basis is None:
            leakage_basis = Infrastructure.leakage_friendly_basis()

        gm_basis = mdl.basis
        new_mdl = mdl.copy()
        
        rho = mdl.preps["rho0"].to_dense()
        rho_new = pgt.change_basis(rho, gm_basis, leakage_basis)
        new_mdl.preps["rho0"] = pgmm.states.FullState(rho_new)

        M0 = mdl.povms["Mdefault"]["0"].to_dense()
        M1 = mdl.povms["Mdefault"]["1"].to_dense()
        new_mdl.povms["Mdefault"] = pgmm.povms.UnconstrainedPOVM(
            [
                ("0", pgt.change_basis(M0, gm_basis, leakage_basis)),
                ("1", pgt.change_basis(M1, gm_basis, leakage_basis)),
            ],
            evotype="default",
        )

        for lbl, op in mdl.operations.items():
            op = op.to_dense()
            op_new = pgt.change_basis(op, gm_basis, leakage_basis)
            new_mdl.operations[lbl] = pgmm.operations.FullArbitraryOp(op_new)
        new_mdl.basis = leakage_basis
        return new_mdl

    @staticmethod
    def changebasis_3level_results(results : ModelEstimateResults):
        """ 
        Return a copy of "results" that changes the basis for every Model within "results"
        into the leakage-friendly basis.
        
        This is needed for report generation.
        """
        results = results.copy()
        leakage_basis = Infrastructure.leakage_friendly_basis()
        for estlbl, est in results.estimates.items():
            for mlbl, mdl in est.models.items():
                if isinstance(mdl, (list, tuple)):  # assume a list/tuple of models
                    new_mdl = [Infrastructure.changebasis_3level_model(m, leakage_basis) for m in mdl]
                else:
                    new_mdl = Infrastructure.changebasis_3level_model(mdl, leakage_basis)
                est.models[mlbl] = new_mdl
            for gopsuitedict_or_list_thereof in est.goparameters.values():
                if not isinstance(gopsuitedict_or_list_thereof, list):
                    gopsuite_listofdicts = [gopsuitedict_or_list_thereof]
                else:
                    gopsuite_listofdicts = gopsuitedict_or_list_thereof
                for gopsuite_dict in gopsuite_listofdicts:
                    m = gopsuite_dict['target_model']
                    m = Infrastructure.changebasis_3level_model(m, leakage_basis)
                    gopsuite_dict['target_model'] = m
            results.estimates[estlbl] = est
        return results

In [3]:
def leaky_qubit_model_from_pspec(ps_2level: QubitProcessorSpec = None):
    """
    Return an ExplicitOpModel for a three-level extension of the two-level system specified by ps_2level.

        The model's POVM has two effects. The first effect is rank-1 and is parallel to the ground state 
        (the first effect being rank-1 might not be appropriate in all situations). The second POVM effect is
        just the identity matrix minus the first effect; this should always be the case for a 2-element POVM.

    This function might be called in a workflow like the following:

        from pygsti.models     import create_explicit_model
        from pygsti.algorithms import find_fiducials, find_germs
        from pygsti.protocols  import StandardGST, StandardGSTDesign, ProtocolData

        # Step 1: Make the experiment design for the 1-qubit system.
        tm_2level = create_explicit_model( ps_2level, ideal_spam_type='CPTPLND', ideal_gate_type='CPTPLND' )
        fids    = find_fiducials( tm_2level )
        germs   = find_germs( tm_2level )
        lengths = [1, 2, 4, 8, 16, 32]
        design  = StandardGSTDesign( tm_2level, fids[0], fids[1], germs, lengths )   
        
        # Step 2: ... run the experiment specified by "design"; store results in a directory "dir" ...

        # Step 3: read in the experimental data and run GST.
        pd  = ProtocolData.from_dir(dir)
        tm_3level = leaky_qubit_model_from_pspec( ps_2level )
        gst = StandardGST( modes=('CPTPLND',), target_model=tm_3level, verbosity=4 )
        res = gst.run(pd)

    Note that we run GST on a 3-level system even though the data was nominally generated by a 2-level system.
    Later cells in this notebook provide functions to take a "res" object from that workflow and run leakage-aware
    gauge optimization plus report generation.
    """
    if ps_2level is None:
        from pygsti.modelpacks import smq1Q_XYI
        ps_2level = smq1Q_XYI.target_model().create_processor_spec()
    
    assert ps_2level.num_qubits == 1
    
    Us = ps_2level.gate_unitaries
    rho0 = np.array([[1, 0, 0], [0, 0, 0], [0, 0, 0]], complex)
    E0   = rho0.copy()
    E1   = np.eye(3, dtype=complex) - E0

    ss = ExplicitStateSpace([0],[3])
    tm_3level = ExplicitOpModel(ss, 'gm')
    tm_3level['rho0'] =  pgt.stdmx_to_gmvec(rho0)
    tm_3level['Mdefault'] = TPPOVM(
        [("0", pygsti.tools.stdmx_to_gmvec(E0)), ("1", pygsti.tools.stdmx_to_gmvec(E1))], evotype="default",
    )

    def u2x2_to_9x9_gm_superoperator(u2x2):
        u3x3 = np.eye(3, dtype=np.complex128)
        u3x3[:2,:2] = u2x2
        superop_std = pgt.unitary_to_std_process_mx(u3x3)
        superop_gm = pgt.change_basis(superop_std, 'std', 'gm')
        return superop_gm

    for gatename, unitary in Us.items():
        gatekey = (gatename, 0) if gatename != '{idle}' else ('Gi',0)
        tm_3level[gatekey] = u2x2_to_9x9_gm_superoperator(unitary)

    tm_3level.sim = 'map'  # can use 'matrix', if that's preferred for whatever reason.
    return tm_3level


In [4]:
def add_lago_estimate(results: ModelEstimateResults, est_key: str, vebosity: int = 0):
    """ 
    Add a key-value pair ('LAGO-' + est_key, new_est) to results.estimates, where
    new_est is obtained by copying results.estimates[est_key] and then applying
    leakage-aware gauge optimization (LAGO) to
    
        existing_mdl := results.estimates[est_key].models['stdgaugeopt'].

    Parameters for the gauge optimization (such as the target model) are obtained
    by making minimal modifications to results.estimates[est_key].goparameters.
    """
    existing_est = results.estimates[est_key].copy()
    new_est_key  = 'LAGO-' + est_key
    results.add_estimate(existing_est, new_est_key)
    gop_params   = existing_est.goparameters
    gop_params   = copy.deepcopy(gop_params)
    for inner_dict in gop_params['stdgaugeopt']:
        inner_dict['method'] = 'L-BFGS-B'
        inner_dict['n_leak'] = 1
        inner_dict['gates_metric'] = 'frobenius squared'
        inner_dict['spam_metric']  = 'frobenius squared'

    existing_mdl = existing_est.models['stdgaugeopt']
    _add_gauge_opt(results, new_est_key, gop_params, existing_mdl, verbosity=vebosity)
    return

from typing import Union, Dict

def write_leakage_friendly_html_report(
        report_title: str,
        report_dir  : str,
        results : Union[ModelEstimateResults, Dict[str,ModelEstimateResults]],
        est_key: str ='CPTPLND',
        gop_verbosity : int = 0
    ):
    if isinstance(results, ModelEstimateResults):
        add_lago_estimate(results, est_key, gop_verbosity)
        results_lfb = Infrastructure.changebasis_3level_results(results)
    else:
        assert isinstance(results, dict)
        assert len(results) > 0
        results_lfb = dict()
        for k, v in results.items():
            add_lago_estimate(v, est_key, gop_verbosity)
            results_lfb[k] = Infrastructure.changebasis_3level_results(v)

    from pygsti.report import construct_standard_report
    report = construct_standard_report(results_lfb, title=report_title, advanced_options={'n_leak': 1})
    report.write_html(report_dir)
    return